<h1 align="center">Python types, hints and annotations</h1>
<h3 align="center">Alexander Hagerman</h3>

<h3 align="center">What are type hints?</h3>

Type hints are annotations on objects to hint to the analyzer and reader what type an object is.

```python
from typing import List
Vector = List[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

# typechecks; a list of floats qualifies as a Vector.
new_vector = scale(2.0, [1.0, -4.2, 5.4])
```

<h4 align="center">Python is dynamic, why would I want to use type hints?</h4>

- Increase code readability and information
- Help catch common bugs
- Better static analysis

**Side note**:
- A dynamic language can still have types. Python is [strongly typed](https://pythonconquerstheuniverse.wordpress.com/2009/10/03/static-vs-dynamic-typing-of-programming-languages/) which means that your objects have types even if you're not making use of that knowledge.

```python
>>> a = 1
>>> type(a)
<class 'int'>
```

<h3 align="center">Adding information</h3>

**Without annotations**

```python
import asyncpg

async def load_events(ids, dbconfig):
    conn = await asyncpg.connect(dbconfig)

    async for id in ids:
        async with conn.transaction():
            rows = await connection.fetchval('select first_name, surname, dob, id, patient_id, events from patient_history where patient_id = $1', id)
            row: PatientRecord
            for row in rows:
                ...
```

**With annotations**

```python
from typing import List
from datetime import datetime
from mypy_extensions import TypedDict
from .patient import Event

class PatientRecord(TypedDict):
    first_name: str
    surname: str
    dob: datetime
    location: int
    id: str
    patient_id: int
    events: List[Event]

```

```python

async def load_events(ids, dbconfig: DatabaseConfig):
    conn = await asyncpg.connect(dbconfig)

    id: int
    async for id in ids:
        async with conn.transaction():
            rows = await connection.fetchval('select first_name, surname, dob, location, id, patient_id, events from patient_history where patient_id = $1', id)
            row: PatientRecord
            for row in rows:
               ...
```

<h3 align="center">Cleaning up types and finding bugs</h3>

In [3]:
%%typecheck

from typing import List, Tuple, Dict, Sequence, Optional, Generator

Meetup = Tuple[str, Dict[str, Sequence[str]]]

def meetup_index(meetups: List[Tuple[str, Dict[str, Sequence[str]]]], meetup: str):
    """
    Find the index for a given meetup in a list of meetups. Assumed meetup names are unique.
    """
    index = (i for i, tupl in enumerate(meetups) if tupl[0] == meetup)
    return next(index)


meetups = [('DerbyPy', {'location': 'Sullivan University', 
                        'events': ['https://www.meetup.com/derbypy/events/jgtjnpyzmbjc/',
                                   'https://www.meetup.com/derbypy/events/jgtjnpyznbgc/']}), 
           ('Derby DevOps', {'location': 'Downtown', 
                             'events': ['https://www.meetup.com/DevOps-Louisville/events/261959129/']})]


meetup = meetup_index(meetups, "DerbyPy")

if meetup:
    print(meetup)
else:
    print("Meetup not found")

Meetup not found


In [5]:
%%typecheck 

from typing import List, Tuple, Dict, Sequence, Optional

Meetup = Tuple[str, Dict[str, Sequence[str]]]

def meetup_index(meetups: List[Meetup], meetup: str) -> Optional[int]:
    """
    Find the index for a given meetup in a list of meetups. Assumed meetup names are unique.
    """
    index = (i for i, tupl in enumerate(meetups) if tupl[0] == meetup)
    return next(index)


meetups = [('DerbyPy', {'location': 'Sullivan University', 
                        'events': ['https://www.meetup.com/derbypy/events/jgtjnpyzmbjc/',
                                   'https://www.meetup.com/derbypy/events/jgtjnpyznbgc/']}), 
           ('Derby DevOps', {'location': 'Downtown', 
                             'events': ['https://www.meetup.com/DevOps-Louisville/events/261959129/']})]

meetup = meetup_index(meetups, "DerbyPy")

if meetup >= 0 and meetup is not None:
#if meetup is not None and meetup >= 0:
    print(meetup)
else:
    print("Meetup not found")

0


<h3 align="center">Catching common bugs</h3>

```python
import os
import sqlite3

dsn = os.environ.get(dsn)
conn = sqlite3.connect(dsn)
```

<h3 align="center">Unhandled optional values</h3>

In [33]:
%%typecheck

import os
import sqlite3

dsn = os.environ.get(dsn)
conn = sqlite3.connect(dsn)

<string>:6: error: Cannot determine type of 'dsn'
<string>:8: error: Argument 1 to "connect" has incompatible type "Optional[str]"; expected "Union[bytes, str, _PathLike[str]]"



NameError: name 'dsn' is not defined

In [4]:
#%%typecheck

meetups = {'DerbyPy': {'location': 'Sullivan University', 
                        'events': ['https://www.meetup.com/derbypy/events/jgtjnpyzmbjc/',
                                   'https://www.meetup.com/derbypy/events/jgtjnpyznbgc/']},
           'Derby DevOps': {'location': 'Downtown', 
                             'events': ['https://www.meetup.com/DevOps-Louisville/events/261959129/']}
           }

meetup = meetups.get('JUGGL')

for events in meetup.get('events'):
    for event in events:
        print(event)

AttributeError: 'NoneType' object has no attribute 'get'

<h3 align="center">Incompatible Types</h3>

In [53]:
%%typecheck

import datetime
import sqlite3

def append_retrieve_date(row):
    retrieved = datetime.datetime.now()
    row.append(retrieved)
    return row

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("select 1 as a")
row = cur.fetchone()

updated_row = append_retrieve_date(row)

AttributeError: 'tuple' object has no attribute 'append'

In [54]:
%%typecheck

import datetime
import sqlite3
from typing import List, Tuple

def append_retrieve_date(row: List):
    retrieved = datetime.datetime.now()
    row.append(retrieved)
    return row

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("select 1 as a")
row: Tuple = cur.fetchone()

updated_row = append_retrieve_date(row)

<string>:17: error: Argument 1 to "append_retrieve_date" has incompatible type "Tuple[Any, ...]"; expected "List[Any]"



AttributeError: 'tuple' object has no attribute 'append'

In [55]:
%%typecheck

import datetime
import sqlite3
from typing import List, Tuple

def append_retrieve_date(row: List) -> List:
    retrieved = datetime.datetime.now()
    row.append(retrieved)
    return row

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("select 1 as a")
row: Tuple = cur.fetchone()

updated_row = append_retrieve_date(list(row))

print(updated_row)

[1, datetime.datetime(2019, 8, 22, 9, 57, 26, 707298)]


<h3 align="center">Python annotations _COULD_ be used to help with performance</h3>

- [mypyc](https://mypy-lang.blogspot.com/2019/01/mypy-0660-released.html)
- [cython type hints](http://docs.cython.org/en/latest/src/tutorial/pure.html#static-typing)

Note from PEP 484

```
While the proposed typing module will contain some building blocks for runtime type checking -- in particular the get_type_hints() function -- third party packages would have to be developed to implement specific runtime type checking functionality, for example using decorators or metaclasses. Using type hints for performance optimizations is left as an exercise for the reader.

It should also be emphasized that Python will remain a dynamically typed language, and the authors have no desire to ever make type hints mandatory, even by convention.
```

<h3 align="center">Additional Reading</h3>

- [Python Docs - Typing Module](https://docs.python.org/3/library/typing.html)
- [PEP 484 Type Hints](https://www.python.org/dev/peps/pep-0484/)
- [Python 3.8 what's new in typing](https://docs.python.org/3.8/whatsnew/3.8.html#typing)
    
    


<h3 align="center">Additional Reading</h3>

- [A deep dive on Python type hints - Vicki Boykis](https://veekaybee.github.io/2019/07/08/python-type-hints)
- [Real Python - Type Checking](https://realpython.com/python-type-checking/)
- [What are Type Hints](https://stackoverflow.com/questions/32557920/what-are-type-hints-in-python-3-5)


- [Classic Computer Science Problems with Python](https://www.manning.com/books/classic-computer-science-problems-in-python)
- [Data Science from Scratch Second Edition](https://www.oreilly.com/library/view/data-science-from/9781492041122/)

<h3 align="center">Tools</h3>

- [mypy](http://mypy-lang.org/)
- [pyannotate](https://github.com/dropbox/pyannotate)
- [pytype](https://google.github.io/pytype/) - platform
- [pyre](https://pyre-check.org/) - external deps, platform
- [pyright](https://github.com/microsoft/pyright) - external deps, npm install